## Longcat treniranje

Ova bilježnica samo istrenira longcat mrežu.

(Dolje je treniranje prekinuto jer sam skužio da mreža više ništa ne uči u tim kasnijim epohama, da se može komotno uzeti prvih osam epoha.)

In [1]:
!pip install torchsummary

In [2]:
import torch
import torchvision
from torchvision import transforms
from torchsummary import summary
import json
import pickle

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
model_name = "longcat"

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = torchvision.datasets.ImageFolder(root='dataset/train', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='dataset/test', transform=transform)


In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=64, shuffle=True, num_workers=4)

In [7]:
print(len(train_loader))

5836


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class LongcatNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.conv5_bn = nn.BatchNorm2d(36)
        self.conv5 = nn.Conv2d(36, 36, 3)
  
        self.conv6_bn = nn.BatchNorm2d(36)
        self.conv6 = nn.Conv2d(36, 49, 3)

        self.conv7_bn = nn.BatchNorm2d(49)
        self.conv7 = nn.Conv2d(49, 49, 3)
        
        self.conv8_bn = nn.BatchNorm2d(49)
        self.conv8 = nn.Conv2d(49, 49, 3)
        
        self.conv9_bn = nn.BatchNorm2d(49)
        self.conv9 = nn.Conv2d(49, 49, 3)
        self.pool9 = nn.MaxPool2d(2, 2)

        self.conv10_bn = nn.BatchNorm2d(49)
        self.conv10 = nn.Conv2d(49, 49, 3)
        self.pool10 = nn.MaxPool2d(2, 2)

        self.fc = nn.Linear(1764, 4)

    def forward(self, x):
        x = self.bn1(x)
        x = self.conv2_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv3_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv4_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.conv5_bn(self.pool4(F.relu(self.conv4(x))))

        x = self.conv6_bn(F.relu(self.conv5(x)))  
        x = self.conv7_bn(F.relu(self.conv6(x)))
        x = self.conv8_bn(F.relu(self.conv7(x)))
        x = self.conv9_bn(F.relu(self.conv8(x)))
        
        
        x = self.conv10_bn(self.pool9(F.relu(self.conv9(x))))
        x = self.pool10(F.relu(self.conv10(x)))
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = self.fc(x)
        return x


model = Net()
model = model.to(device)

In [9]:
summary(model, (3, 640, 640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [-1, 3, 640, 640]               6
            Conv2d-2          [-1, 9, 638, 638]             252
         MaxPool2d-3          [-1, 9, 319, 319]               0
       BatchNorm2d-4          [-1, 9, 319, 319]              18
            Conv2d-5         [-1, 16, 317, 317]           1,312
         MaxPool2d-6         [-1, 16, 158, 158]               0
       BatchNorm2d-7         [-1, 16, 158, 158]              32
            Conv2d-8         [-1, 25, 156, 156]           3,625
         MaxPool2d-9           [-1, 25, 78, 78]               0
      BatchNorm2d-10           [-1, 25, 78, 78]              50
           Conv2d-11           [-1, 36, 76, 76]           8,136
        MaxPool2d-12           [-1, 36, 38, 38]               0
      BatchNorm2d-13           [-1, 36, 38, 38]              72
           Conv2d-14           [-1, 36,

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.3)

In [11]:
early_batches_acc = {0, 2, 4, 6, 8, 10, 15, 20, 30, 50,
                                75, 100, 150, 200, 300, 400, 500,
                               600, 700, 800}

#torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_0_batch_0.pth')

In [12]:
model.load_state_dict(torch.load("saved_models/longcat/epoch_1_batch_2999.pth"))

<All keys matched successfully>

In [13]:
test_accs = dict()

for epoch in range(1,10):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if (epoch == 1) and (i < 2999):
            continue
        if (i % 1000 == 1) or (epoch == 0 and (i in early_batches_acc)):
            correct = 0
            total = 0
            with torch.no_grad():
                for ix, data in enumerate(test_loader):
                    if ix > 100:
                        break
                    images, labels = data
                    images, labels = images.cuda(), labels.cuda()
                    # calculate outputs by running images through the network
                    outputs = model(images)
                    # the class with the highest energy is what we choose as prediction
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            acc = correct / total
            test_accs[(epoch, i)] = acc
            print(f'Accuracy of the network on the test images: {100*acc} %')
            with open(f'accuracies_{model_name}.pickle', 'wb') as fp:
                pickle.dump(test_accs, fp)
        if epoch == 0 and (i % 2 == 1):
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')
        
        if epoch == 1 and (i % 2 == 1):
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        if epoch > 1 and i % 40 == 0:
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
    
    if epoch > 0:
        scheduler.step()

print('Finished Training')

[2,  3000] loss: 0.002
Accuracy of the network on the test images: 81.26547029702971 %
[2,  3200] loss: 0.245
[2,  3400] loss: 0.256
[2,  3600] loss: 0.305
[2,  3800] loss: 0.228
[2,  4000] loss: 0.226
Accuracy of the network on the test images: 82.45668316831683 %
[2,  4200] loss: 0.206
[2,  4400] loss: 0.225
[2,  4600] loss: 0.212
[2,  4800] loss: 0.224
[2,  5000] loss: 0.198
Accuracy of the network on the test images: 80.73948019801979 %
[2,  5200] loss: 0.183
[2,  5400] loss: 0.189
[2,  5600] loss: 0.194
[2,  5800] loss: 0.206
Accuracy of the network on the test images: 83.86448019801979 %
[3,   200] loss: 0.144
[3,   400] loss: 0.127
[3,   600] loss: 0.151
[3,   800] loss: 0.134
[3,  1000] loss: 0.129
Accuracy of the network on the test images: 85.10210396039604 %
[3,  1200] loss: 0.118
[3,  1400] loss: 0.113
[3,  1600] loss: 0.111
[3,  1800] loss: 0.121
[3,  2000] loss: 0.108
Accuracy of the network on the test images: 86.01485148514851 %
[3,  2200] loss: 0.120
[3,  2400] loss: 0

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
confusion_matrix = torch.zeros(4, 4)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
!nvidia-smi

In [ ]:
confusion_matrix